In [1]:
import json
import csv

In [2]:
with open("./parse-traceroutes-output.json") as f:
	json_dict = json.load(f)

tr_methods = list(json_dict.keys())

tr_methods

['udp', 'icmp', 'tcp', 'paris', 'dublin']

In [3]:
def get_last_successful_hop_data(hops):
	"""Extracts the hop number, lowest rtt, and corresponding IP from last successful hop."""
	for hop in reversed(hops):
		rtts = [str(t).rstrip('ms') for t in hop['RTTs']]
		if ''.join(rtts) in ('*', '***'):
			continue
		
		rtts = [float(t) for t in rtts if t != '*']
		ips = [ip for ip in hop['HopIPs'] if ip != '*']
		
		best_rtt = min(rtts)
		best_ip = ips[rtts.index(best_rtt)]
		hop_num = hop['TTL']

		return hop_num, best_ip, best_rtt


In [4]:
results = {}

for tr in tr_methods:
	results[tr] = {}
	tr_data = json_dict[tr]

	for tr_dict in tr_data:
		dest_ip = tr_dict['DestinationIP']
		hops = tr_dict['Hops']
		last_hop, last_ip, last_rtt = get_last_successful_hop_data(hops)
		results[tr][dest_ip] = {'last_hop': last_hop, 'last_ip': last_ip, 'last_rtt': last_rtt}


In [5]:
for tr, res in results.items():
	print(f'{tr:8}: {len(res.keys())} data points')

udp     : 985 data points
icmp    : 985 data points
tcp     : 985 data points
paris   : 985 data points
dublin  : 975 data points


In [6]:
ips = list(results['dublin'].keys())

with open ('data.csv', 'w', newline='') as f:
	writer = csv.writer(f)
	writer.writerow(['Detination IP', 'Type', 'Last Hop #', 'Last IP', 'Last RTT'])
	for ip in ips:
		for tr in tr_methods:
			row = results[tr][ip]
			writer.writerow([ip, tr, row['last_hop'], row['last_ip'], row['last_rtt']])
